LDA_bertfor3levels

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings

warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/LDA_on_embeddings/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/LDA_on_embeddings/bert_2lda_withstopwordchangelevels_2_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1
0,0,-0.345479,-2.260875
1,1,-0.225225,-1.212315
2,2,3.959834,0.560789
3,3,-2.043962,-1.112190
4,4,1.273862,-2.475487


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1
0,-0.345479,-2.260875
1,-0.225225,-1.212315
2,3.959834,0.560789
3,-2.043962,-1.112190
4,1.273862,-2.475487


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 2), (524, 2))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.33260886017255314


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.455738,0.455738,0.464574,0.44718,0.354962,0.354962,0.328651,0.334288


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2564    0.2548    0.2556       157
           2     0.4491    0.5401    0.4904       237
           3     0.2651    0.1692    0.2066       130

    accuracy                         0.3626       524
   macro avg     0.3235    0.3214    0.3175       524
weighted avg     0.3457    0.3626    0.3496       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.455738,0.455738,0.464574,0.44718,0.354962,0.354962,0.328651,0.334288
1,RandomForest,1.000000,1.000000,1.000000,1.00000,0.362595,0.362595,0.345718,0.349641


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5110    0.2576    0.3425       361
           2     0.5083    0.8879    0.6465       553
           3     0.4583    0.1078    0.1746       306

    accuracy                         0.5057      1220
   macro avg     0.4925    0.4178    0.3879      1220
weighted avg     0.4966    0.5057    0.4382      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2235    0.1210    0.1570       157
           2     0.4450    0.7679    0.5635       237
           3     0.1667    0.0385    0.0625       130

    accuracy                         0.3931       524
   macro avg     0.2784    0.3091    0.2610       524
weighted avg     0.3096    0.3931    0.3174       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.504918,0.504918,0.524038,0.436498,0.408397,0.408397,0.325646,0.313816
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.358779,0.358779,0.344490,0.349043
2,Adaboost,0.505738,0.505738,0.496555,0.438187,0.393130,0.393130,0.309586,0.317404


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.7611    0.5208    0.6184       361
           2     0.6070    0.9439    0.7389       553
           3     0.8673    0.3203    0.4678       306

    accuracy                         0.6623      1220
   macro avg     0.7451    0.5950    0.6084      1220
weighted avg     0.7179    0.6623    0.6352      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2685    0.1847    0.2189       157
           2     0.4599    0.7511    0.5705       237
           3     0.3103    0.0692    0.1132       130

    accuracy                         0.4122       524
   macro avg     0.3463    0.3350    0.3009       524
weighted avg     0.3655    0.4122    0.3517       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.504918,0.504918,0.524038,0.436498,0.408397,0.408397,0.325646,0.313816
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.358779,0.358779,0.344490,0.349043
2,Adaboost,0.505738,0.505738,0.496555,0.438187,0.393130,0.393130,0.309586,0.317404
3,GBM,0.662295,0.662295,0.717875,0.635227,0.412214,0.412214,0.365477,0.351700


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3198    0.5873    0.4141       361
           2     0.4720    0.2134    0.2939       553
           3     0.2671    0.2680    0.2675       306

    accuracy                         0.3377      1220
   macro avg     0.3530    0.3562    0.3252      1220
weighted avg     0.3756    0.3377    0.3228      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3018    0.5287    0.3843       157
           2     0.5000    0.2447    0.3286       237
           3     0.2406    0.2462    0.2433       130

    accuracy                         0.3302       524
   macro avg     0.3475    0.3398    0.3187       524
weighted avg     0.3763    0.3302    0.3241       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.455738,0.455738,0.464574,0.447180,0.354962,0.354962,0.328651,0.334288
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.362595,0.362595,0.345718,0.349641
2,SVM,0.337705,0.337705,0.375559,0.322843,0.330153,0.330153,0.376267,0.324131


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5147    0.5319    0.5232       361
           2     0.5944    0.7342    0.6570       553
           3     0.5549    0.2974    0.3872       306

    accuracy                         0.5648      1220
   macro avg     0.5547    0.5211    0.5225      1220
weighted avg     0.5609    0.5648    0.5497      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3046    0.3376    0.3202       157
           2     0.4437    0.5316    0.4837       237
           3     0.2576    0.1308    0.1735       130

    accuracy                         0.3740       524
   macro avg     0.3353    0.3333    0.3258       524
weighted avg     0.3558    0.3740    0.3578       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.504918,0.504918,0.524038,0.436498,0.408397,0.408397,0.325646,0.313816
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.358779,0.358779,0.344490,0.349043
2,Adaboost,0.505738,0.505738,0.496555,0.438187,0.393130,0.393130,0.309586,0.317404
3,GBM,0.662295,0.662295,0.717875,0.635227,0.412214,0.412214,0.365477,0.351700
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.564754,0.564754,0.560934,0.549715,0.374046,0.374046,0.355829,0.357753


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4600    0.0637    0.1119       361
           2     0.4623    0.9765    0.6275       553
           3     0.5000    0.0033    0.0065       306

    accuracy                         0.4623      1220
   macro avg     0.4741    0.3478    0.2487      1220
weighted avg     0.4711    0.4623    0.3192      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.1579    0.0191    0.0341       157
           2     0.4444    0.9451    0.6046       237
           3     0.0000    0.0000    0.0000       130

    accuracy                         0.4332       524
   macro avg     0.2008    0.3214    0.2129       524
weighted avg     0.2483    0.4332    0.2837       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores) 
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.504918,0.504918,0.524038,0.436498,0.408397,0.408397,0.325646,0.313816
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.358779,0.358779,0.344490,0.349043
2,Adaboost,0.505738,0.505738,0.496555,0.438187,0.393130,0.393130,0.309586,0.317404
3,GBM,0.662295,0.662295,0.717875,0.635227,0.412214,0.412214,0.365477,0.351700
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.564754,0.564754,0.560934,0.549715,0.374046,0.374046,0.355829,0.357753
6,GNB,0.462295,0.462295,0.471088,0.319198,0.433206,0.433206,0.248326,0.283664
